## Initial setup

In [1]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [ ]:
!nvidia-smi

Sun Sep 27 13:11:00 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Install wandb for experiment tracking
!pip install --upgrade https://github.com/wandb/client/archive/feature/code-save.zip

     \ 12.3MB 1.1MB/s
     |████████████████████████████████| 163kB 2.9MB/s 
     |████████████████████████████████| 102kB 5.8MB/s 
     |████████████████████████████████| 122kB 8.7MB/s 
     |████████████████████████████████| 102kB 6.2MB/s 
     |████████████████████████████████| 71kB 6.7MB/s 
     |████████████████████████████████| 71kB 5.5MB/s 
  Created wheel for wandb: filename=wandb-0.8.36-py2.py3-none-any.whl size=1385041 sha256=b44a44a988a409730352bcd4c27fcf7997d782922b0c81e22f8026cef851ae93
  Stored in directory: /root/.cache/pip/wheels/6e/96/54/f4aafa18d112d4c6e313ac393b3b85513d090dd15b37357f19
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=d02a2a1620546ee4a21f4cba00087533c659bd8c1311fd266b8b60d9f5624738
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f0addafb3cbb6bd2da23
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=7cad666285f0e0df597f518e64f40e6ff8ad5fe48236f3

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Other imports
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import matplotlib.pyplot as plt
from imutils import paths
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
# Random seed fixation
tf.random.set_seed(666)
np.random.seed(666)

## Dataset gathering and preparation

In [ ]:
# Gather dataset
#!git clone https://drive.google.com/drive/u/0/folders/1cHUBZXG3roJm09WwiX9GNiXYKTowWLzo
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Datasets/HARDataset/

Mounted at /content/drive/
/content/drive/My Drive/Datasets/HARDataset


In [ ]:
# How many training images for SimCLR?
train_history = pd.read_csv("Aug_HAR_train_x.csv")
train_future = pd.read_csv("Aug_HAR_train_y.csv")
train_lbl = pd.read_csv("Aug_HAR_train_lbl.txt")
train_images = list(paths.list_images("imagenet-5-categories/train"))
print(len(train_history))

7351


In [ ]:
# Augmentation utilities (differs from the original implementation)
# Referred from: https://arxiv.org/pdf/2002.05709.pdf (Appendxi A 
# corresponding GitHub: https://github.com/google-research/simclr/)

class CustomAugment(object):
    def __call__(self, sample):        
        # Random flips
        sample = self._random_apply(tf.image.flip_left_right, sample, p=0.5)
        
        # Randomly apply transformation (color distortions) with probability p.
        sample = self._random_apply(self._color_jitter, sample, p=0.8)
        sample = self._random_apply(self._color_drop, sample, p=0.2)

        return sample

    def _color_jitter(self, x, s=1):
        # one can also shuffle the order of following augmentations
        # each time they are applied.
        x = tf.image.random_brightness(x, max_delta=0.8*s)
        x = tf.image.random_contrast(x, lower=1-0.8*s, upper=1+0.8*s)
        x = tf.image.random_saturation(x, lower=1-0.8*s, upper=1+0.8*s)
        x = tf.image.random_hue(x, max_delta=0.2*s)
        x = tf.clip_by_value(x, 0, 1)
        return x
    
    def _color_drop(self, x):
        x = tf.image.rgb_to_grayscale(x)
        x = tf.tile(x, [1, 1, 1, 3])
        return x
    
    def _random_apply(self, func, x, p):
        return tf.cond(
          tf.less(tf.random.uniform([], minval=0, maxval=1, dtype=tf.float32),
                  tf.cast(p, tf.float32)),
          lambda: func(x),
          lambda: x)

In [ ]:
# Build the augmentation pipeline
data_augmentation = Sequential([Lambda(CustomAugment())])

In [ ]:
# Image preprocessing utils
@tf.function
def parse_images(image_path):
    image_string = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size=[224, 224])

    return image

In [ ]:
!wget https://github.com/shdeldari/SimCLR-in-TensorFlow-2/tree/Time-Series/utils/DataHelper.py
!wget https://raw.githubusercontent.com/shdeldari/SimCLR-in-TensorFlow-2/Time-Series/utils/training_generator.py


--2020-09-27 13:19:29--  https://github.com/shdeldari/SimCLR-in-TensorFlow-2/tree/Time-Series/utils/DataHelper.py
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/shdeldari/SimCLR-in-TensorFlow-2/blob/Time-Series/utils/DataHelper.py [following]
--2020-09-27 13:19:30--  https://github.com/shdeldari/SimCLR-in-TensorFlow-2/blob/Time-Series/utils/DataHelper.py
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘DataHelper.py.2’

DataHelper.py.2         [  <=>               ] 138.03K   259KB/s    in 0.5s    

2020-09-27 13:19:31 (259 KB/s) - ‘DataHelper.py.2’ saved [141347]

--2020-09-27 13:19:31--  https://raw.githubusercontent.com/shdeldari/SimCLR-in-TensorFlow-2/Time-Series/utils/training_generator.py
Resolving raw.githubusercontent.com (raw.gi

In [ ]:
import training_generator as gen

ModuleNotFoundError: ignored

In [ ]:

# Create TensorFlow dataset
BATCH_SIZE = 6
dataset = "Aug_HAR"
data_path = "/content/drive/My Drive/Datasets/HARDataset/"

train_data = gen.semi_hard_training_generator(data_path, dataset, 100, 28, batch_size=BATCH_SIZE,
                                                        mode='train', aug=aug)
validation_data = gen.semi_hard_training_generator(data_path, dataset, 100, 28, batch_size=BATCH_SIZE,
                                                        mode='validation', aug=aug)
"""
train_ds = tf.data.Dataset.from_tensor_slices(train_images)
train_ds = (
    train_ds
    .map(parse_images, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .shuffle(1024)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)
"""

NameError: ignored

## Utilities

In [ ]:
# Architecture utils
def get_resnet_simclr(hidden_1, hidden_2, hidden_3):
    base_model = tf.keras.applications.ResNet50(include_top=False, weights=None, input_shape=(224, 224, 3))
    base_model.trainable = True
    inputs = Input((224, 224, 3))
    h = base_model(inputs, training=True)
    h = GlobalAveragePooling2D()(h)

    projection_1 = Dense(hidden_1)(h)
    projection_1 = Activation("relu")(projection_1)
    projection_2 = Dense(hidden_2)(projection_1)
    projection_2 = Activation("relu")(projection_2)
    projection_3 = Dense(hidden_3)(projection_2)

    resnet_simclr = Model(inputs, projection_3)

    return resnet_simclr

In [ ]:
!wget https://github.com/shdeldari/SimCLR-in-TensorFlow-2/blob/master/helpers.py
!wget https://github.com/shdeldari/SimCLR-in-TensorFlow-2/blob/master/losses.py

--2020-09-27 05:17:05--  https://github.com/shdeldari/SimCLR-in-TensorFlow-2/blob/master/helpers.py
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘helpers.py.2’

helpers.py.2            [ <=>                ]  92.09K  --.-KB/s    in 0.02s   

2020-09-27 05:17:05 (3.94 MB/s) - ‘helpers.py.2’ saved [94303]

--2020-09-27 05:17:05--  https://github.com/shdeldari/SimCLR-in-TensorFlow-2/blob/master/losses.py
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘losses.py.2’

losses.py.2             [ <=>                ]  90.60K  --.-KB/s    in 0.02s   

2020-09-27 05:17:06 (5.25 MB/s) - ‘losses.py.2’ saved [92770]



Don't forget to comment the `augmentation` import in the `helpers.py` script.

In [ ]:
from losses import _dot_simililarity_dim1 as sim_func_dim1, _dot_simililarity_dim2 as sim_func_dim2
#import helpers

In [ ]:
def get_negative_mask(batch_size):
    # return a mask that removes the similarity score of equal/similar images.
    # this function ensures that only distinct pair of images get their similarity scores
    # passed as negative examples
    negative_mask = np.ones((batch_size, 2 * batch_size), dtype=bool)
    for i in range(batch_size):
        negative_mask[i, i] = 0
        negative_mask[i, i + batch_size] = 0
    return tf.constant(negative_mask)


def gaussian_filter(v1, v2):
    k_size = int(v1.shape[1] * 0.1)  # kernel size is set to be 10% of the image height/width
    gaussian_ope = GaussianBlur(kernel_size=k_size, min=0.1, max=2.0)
    [v1, ] = tf.py_function(gaussian_ope, [v1], [tf.float32])
    [v2, ] = tf.py_function(gaussian_ope, [v2], [tf.float32])
    return v1, v2

In [ ]:
# Mask to remove positive examples from the batch of negative samples
negative_mask = get_negative_mask(BATCH_SIZE)
print(negative_mask)

tf.Tensor(
[[False  True  True ...  True  True  True]
 [ True False  True ...  True  True  True]
 [ True  True False ...  True  True  True]
 ...
 [ True  True  True ... False  True  True]
 [ True  True  True ...  True False  True]
 [ True  True  True ...  True  True False]], shape=(64, 128), dtype=bool)


In [ ]:
@tf.function
def train_step(xis, xjs, model, optimizer, criterion, temperature):
    with tf.GradientTape() as tape:
        zis = model(xis)
        zjs = model(xjs)

        # normalize projection feature vectors
        zis = tf.math.l2_normalize(zis, axis=1)
        zjs = tf.math.l2_normalize(zjs, axis=1)

        l_pos = sim_func_dim1(zis, zjs)
        print(l_pos.shape)
        l_pos = tf.reshape(l_pos, (BATCH_SIZE, 1))
        l_pos /= temperature

        negatives = tf.concat([zjs, zis], axis=0)

        loss = 0

        for positives in [zis, zjs]:
            l_neg = sim_func_dim2(positives, negatives)

            labels = tf.zeros(BATCH_SIZE, dtype=tf.int32)

            l_neg = tf.boolean_mask(l_neg, negative_mask)
            l_neg = tf.reshape(l_neg, (BATCH_SIZE, -1))
            l_neg /= temperature

            logits = tf.concat([l_pos, l_neg], axis=1) 
            loss += criterion(y_pred=logits, y_true=labels)

        loss = loss / (2 * BATCH_SIZE)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss

In [ ]:
wandb.init(project="simclr")

wandb: Wandb version 0.10.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


W&B Run: https://app.wandb.ai/shohreh/simclr/runs/86b0wfjm

In [ ]:
def train_simclr(model, dataset, optimizer, criterion,
                 temperature=0.1, epochs=100):
    step_wise_loss = []
    epoch_wise_loss = []

    for epoch in tqdm(range(epochs)):
        for image_batch in dataset:
            a = data_augmentation(image_batch)
            b = data_augmentation(image_batch)

            loss = train_step(a, b, model, optimizer, criterion, temperature)
            step_wise_loss.append(loss)

        epoch_wise_loss.append(np.mean(step_wise_loss))
        wandb.log({"nt_xentloss": np.mean(step_wise_loss)})
        
        if epoch % 10 == 0:
            print("epoch: {} loss: {:.3f}".format(epoch + 1, np.mean(step_wise_loss)))

    return epoch_wise_loss, model

## Training

In [ ]:
criterion = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, 
                                                          reduction=tf.keras.losses.Reduction.SUM)
decay_steps = 1000
lr_decayed_fn = tf.keras.experimental.CosineDecay(
    initial_learning_rate=0.1, decay_steps=decay_steps)
optimizer = tf.keras.optimizers.SGD(lr_decayed_fn)

resnet_simclr_2 = get_resnet_simclr(256, 128, 50)

epoch_wise_loss, resnet_simclr  = train_simclr(resnet_simclr_2, train_ds, optimizer, criterion,
                 temperature=0.1, epochs=200)

with plt.xkcd():
    plt.plot(epoch_wise_loss)
    plt.title("tau = 0.1, h1 = 256, h2 = 128, h3 = 50")
    plt.show()



  0%|          | 0/200 [00:00<?, ?it/s]

(64, 1, 1)


  0%|          | 0/200 [00:04<?, ?it/s]


ValueError: ignored

## Store the weights to a safe place :)

In [ ]:
from google.colab import auth
auth.authenticate_user()

PROJECT_ID = "fast-ai-exploration" 
!gcloud config set project $PROJECT_ID

In [ ]:
import datetime
filename = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "resnet_simclr.h5"
filename

In [ ]:
#!gsutil mb gs://resnet_simclr_imagenet
resnet_simclr.save_weights(filename)
!gsutil -m cp 20200508-134915resnet_simclr.h5 gs://resnet_simclr_imagenet/

## In case saving to a GCS bucket is not possible

In [ ]:
resnet_simclr.save_weights(filename)
wandb.save(filename)